In [70]:
import pandas as pd

In [71]:
df = pd.read_csv("C:/Users/acer/Documents/Data Analyst - Field Training Program/Portfolio/3. E-commerce/Dataset/data.csv", encoding="latin1")

## Key Points

- Customers are mostly wholesalers
- Dataset b/w 01/12/2010-09/12/2011
- The company mainly sells unique all-occasion gifts



### Problem statement & Questions

" How can the company optimize revenue, customer retention, and inventory efficiency by better understanding customer purchasing behavior, product performance, and sales patterns across markets? "


- Which customers generate the most revenue, and how can we increase their loyalty?
- What products are underperforming, and should we discontinue or promote them more heavily?
- How do sales trends vary by country, and what can we do to optimize performance in underperforming regions?
- What is the return rate by product, and how can we reduce returns to improve profitability?
- What are the seasonal buying patterns, and how can we optimize inventory and promotions around these periods?




### Final Outputs Metrics

1. Revenue by Customer & Product
    - Top 10 Customers by Revenue
    - Top 10 Products by Revenue
    - Revenue per Product Category
2. Customer Segmentation & Buying Patterns
    - Customer Lifetime Value (CLV) – Estimated future revenue from each customer.
    - Frequency of Purchase – Average number of purchases per customer within a time period.
    - Average Order Value (AOV) – The average amount spent per order for each customer group.
3. Sales Trends & Seasonality
    - Monthly/Quarterly Sales Trend
    - Seasonal Demand Patterns
4. Product Performance & Inventory Optimization
    - Slow-Moving Products – Products with low sales or excess inventory.
    - Stockouts & Overstock Analysis – Frequency of stockouts for popular products or overstocked items.
5. Sales by Country/Region
    - Total Sales by Country
    - Country Growth Rate – Growth in revenue per country over time, identifying high-potential markets.
    - Regional Market Share – Comparing the company’s sales to overall market potential in different countries.
6. Customer Acquisition and Retention Rates
    - Customer Churn Rate – Percentage of customers who stopped buying after a certain period.
    - New vs. Returning Customer Sales – Revenue split between new customers and repeat buyers.
    - Customer Retention Rate – Percentage of customers who make repeat purchases.   

#### The final outputs could include:

- A Customer Insights Dashboard that shows customer segmentation, purchasing patterns, and lifetime value.
- A Product Performance Report to help identify which products to focus on, promote, or discontinue.
- A Geographic Sales Report highlighting performance across different countries/regions.
- A Revenue Forecasting Model based on sales trends, seasonality, and customer purchasing behavior.
- A Return & Satisfaction Analysis to reduce returns and improve product offerings.




In [72]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## Data Cleaning Plan

This dataset contains transactional sales data with missing values, inconsistent formats, and potential anomalies that must be addressed before analysis. The following steps outline the data cleaning and preprocessing strategy.

---

### 1. Data Type Standardization
- Convert **InvoiceDate** from object to `datetime` format.
- Ensure **CustomerID** is stored as an integer (or nullable integer) instead of float.
- Confirm **Quantity** is an integer and **UnitPrice** is a float.

---

### 2. Handling Missing Values
- **CustomerID**:
  - Remove records with missing CustomerID for customer-level analysis (CLV, retention, segmentation).
  - Retain these records for overall sales and product-level analysis if needed.
- **Description**:
  - Fill missing product descriptions using StockCode where possible.
  - Drop rows with missing descriptions if they cannot be reliably inferred.
  - Checking for rows "POST" or "BANK" and excluding them for product related analysis   

---

### 3. Removing Invalid & Anomalous Records
- Remove transactions with:
  - **Quantity ≤ 0** (indicates returns or data errors).
  - **UnitPrice ≤ 0** (invalid pricing).
- Identify and flag return transactions (negative quantities or invoice numbers starting with “C”) for return rate analysis.

---

### 4. Duplicate Handling
- Identify duplicate rows based on:
  - InvoiceNo
  - StockCode
  - Quantity
  - InvoiceDate
- Remove exact duplicates to avoid revenue inflation.

---

### 5. Outlier Detection & Treatment
- Detect extreme outliers in:
  - Quantity
  - UnitPrice
  - Revenue (Quantity × UnitPrice)
- Cap or remove outliers using statistical methods (IQR or Z-score) depending on business relevance.

---

### 6. Feature Engineering
- Create derived features:
  - **Revenue** = Quantity × UnitPrice
  - **Invoice Month / Quarter / Year** from InvoiceDate
  - **Customer Type** (New vs Returning)
  - **Order Value** per invoice
- Aggregate data at:
  - Customer level (CLV, frequency, AOV)
  - Product level (sales volume, revenue)
  - Country level (regional performance)

---

### 7. Country & Text Standardization
- Standardize **Country** names (case formatting, spelling).
- Clean **Description** text:
  - Remove extra whitespace
  - Convert to uppercase or lowercase consistently

---

### 8. Time-Based Validation
- Confirm all transactions fall within the expected date range:
  - **01/12/2010 – 09/12/2011**
- Remove or flag any records outside this period.

---

### 9. Data Integrity Checks
- Validate:
  - Revenue totals after cleaning
  - Consistency between Quantity, UnitPrice, and Revenue
- Re-run summary statistics after each major cleaning step.

---

### 10. Final Clean Dataset Outputs
- **Clean Transactions Dataset** – Ready for exploratory analysis
- **Customer-Level Dataset** – For segmentation, CLV, retention analysis
- **Product-Level Dataset** – For inventory and performance optimization
- **Country-Level Dataset** – For geographic and market analysis

---

This cleaning process ensures the dataset is reliable, consistent, and optimized for advanced analytics, forecasting, and business decision-making.


#### Cleaning 1.1

In [74]:
# Cleaning 1.1

df["InvoiceDate"] = pd.to_datetime(df['InvoiceDate'])



In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


#### Cleaning 1.2

In [76]:
# Cleaning 1.2

df['CustomerID'] = df['CustomerID'].astype("Int64")



In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  Int64         
 7   Country      541909 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 33.6+ MB


In [78]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


#### Cleaning 2.1

In [79]:
# Cleaning 2.1

df_cus = df.dropna(subset=["CustomerID"])


In [80]:
df_cus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  Int64         
 7   Country      406829 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 28.3+ MB


#### Cleaning 2.2

In [81]:
# Cleaning 2.2

desc_check = (
    df.dropna(subset=["Description"])
      .groupby("StockCode")["Description"]
      .nunique()
      .sort_values(ascending=False)
)

desc_check.head(10)




StockCode
20713     8
23084     7
21830     6
85175     6
21181     5
23131     5
23343     5
72807A    5
85172     5
21621     4
Name: Description, dtype: int64

In [ ]:
stock_desc_map = (
    df.dropna(subset=["Description"])
      .groupby("StockCode")["Description"]
      .agg(lambda x: x.mode().iloc[0])
)

In [83]:
stock_desc_map.head()

StockCode
10002     INFLATABLE POLITICAL GLOBE 
10080        GROOVY CACTUS INFLATABLE
10120                    DOGGY RUBBER
10123C          HEARTS WRAPPING TAPE 
10124A    SPOTS ON RED BOOKCOVER TAPE
Name: Description, dtype: object

In [90]:
# Flaggin filled description
df["DescriptionFilled"] = df["Description"].isna()



In [92]:
df["Description"] = df["Description"].fillna(
    df["StockCode"].map(stock_desc_map)
)


In [93]:
df["Description"].isna().sum()


np.int64(112)

In [97]:
# Dropping rows with na description

df = df.dropna(subset=['Description'])
df["Description"].isna().sum()


np.int64(0)

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 541797 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   InvoiceNo          541797 non-null  object        
 1   StockCode          541797 non-null  object        
 2   Description        541797 non-null  object        
 3   Quantity           541797 non-null  int64         
 4   InvoiceDate        541797 non-null  datetime64[ns]
 5   UnitPrice          541797 non-null  float64       
 6   CustomerID         406829 non-null  Int64         
 7   Country            541797 non-null  object        
 8   DescriptionFilled  541797 non-null  bool          
dtypes: Int64(1), bool(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 38.2+ MB


In [98]:
# Filter out these rows for product-based analysis
df_prod_only = df[~df["StockCode"].str.contains('POST|BANK|SHIPPING|DISCOUNT|LOYALTY', case=False, na=False)]
df_prod_only.info()

<class 'pandas.core.frame.DataFrame'>
Index: 540504 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   InvoiceNo          540504 non-null  object        
 1   StockCode          540504 non-null  object        
 2   Description        540504 non-null  object        
 3   Quantity           540504 non-null  int64         
 4   InvoiceDate        540504 non-null  datetime64[ns]
 5   UnitPrice          540504 non-null  float64       
 6   CustomerID         405621 non-null  Int64         
 7   Country            540504 non-null  object        
 8   DescriptionFilled  540504 non-null  bool          
dtypes: Int64(1), bool(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 38.1+ MB


In [87]:
Prod_only_df = df[~df["StockCode"].str.contains('POST|BANK|SHIPPING|DISCOUNT|LOYALTY', case=False, na=False)]


In [88]:
Prod_only_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 540616 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    540616 non-null  object        
 1   StockCode    540616 non-null  object        
 2   Description  539166 non-null  object        
 3   Quantity     540616 non-null  int64         
 4   InvoiceDate  540616 non-null  datetime64[ns]
 5   UnitPrice    540616 non-null  float64       
 6   CustomerID   405621 non-null  Int64         
 7   Country      540616 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 37.6+ MB
